**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
suppressMessages(suppressWarnings(library("DESeq2")))
suppressMessages(suppressWarnings(library("edgeR")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



## Loop through regions and assays to create count matrix

**Check files: region**

In [2]:
txt_fdiry = file.path(FD_RES, "region_coverage_fcc")

vec = dir(txt_fdiry)
for (txt in vec){cat(txt, "\n")}

atac_ENCFF333TAT 
atac_ENCFF558BLC 
atac_ENCFF925CYR 
atac_ENCFF948AFM 
dnase_ENCFF185XRG 
dnase_ENCFF274YGF 
fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


**Check files: assays**

In [3]:
txt_assay  = "MPRA_Tiling_K562_Tewhey_Hannah"
txt_region = "fcc_astarr_macs_input_overlap"
txt_fdiry  = file.path(
    FD_RES, "region_coverage_fcc",
    txt_region,
    txt_assay,
    "overlap_count",
    "summary"
)
txt_fname = "data*raw*rds"
txt_fglob = file.path(txt_fdiry, txt_fname)

vec_txt_fpath = Sys.glob(txt_fglob)
print(vec_txt_fpath)

[1] "/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/MPRA_Tiling_K562_Tewhey_Hannah/overlap_count/summary/data.count_column.raw.OL13.rds"
[2] "/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/MPRA_Tiling_K562_Tewhey_Hannah/overlap_count/summary/data.count_column.raw.OL43.rds"
[3] "/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_overlap/MPRA_Tiling_K562_Tewhey_Hannah/overlap_count/summary/data.count_column.raw.OL45.rds"


**Generate effect size**

In [4]:
### init: region folders
txt_fdiry = file.path(FD_RES, "region_coverage_fcc")
vec_txt_region = dir(txt_fdiry)

### init: assay name
txt_assay  = "MPRA_Tiling_K562_Tewhey_Hannah"

### init: assay prefix and helper function
vec_txt_prefix = c("OL13", "OL43", "OL45")
fun_get_prefix = function(txt){
    res = fun_str_map_detect(txt, vec_txt_prefix, vec_txt_prefix)
    return(res)
}

### loop
for (txt_region in vec_txt_region) {

    ### init: get RDS files w/ count matrix and column information
    txt_fdiry = file.path(FD_RES, "region_coverage_fcc", txt_region, txt_assay, "overlap_count", "summary")
    txt_fname = "data*count_column*raw*rds"
    txt_fglob = file.path(txt_fdiry, txt_fname)
    
    vec_txt_fpath_inp = Sys.glob(txt_fglob)
    
    ### loop
    for (txt_fpath_inp in vec_txt_fpath_inp){

        ### read object
        lst_dat = readRDS(txt_fpath_inp)
        dat_cnt = lst_dat$data_cnt
        dat_col = lst_dat$data_col
        
        ### arrange column data
        dat_col = dat_col  %>% 
            dplyr::mutate(Group = factor(Group, levels = c("Input", "Output"))) %>%
            column_to_rownames(var = "Sample")
        
        ### arrange count matrix
        mat_tot = dat_cnt %>%
            dplyr::select(-Chrom, -ChromStart, -ChromEnd) %>%
            column_to_rownames(var = "Region")
        mat_inp = mat_tot %>% dplyr::select(starts_with("Input"))
        mat_out = mat_tot %>% dplyr::select(starts_with("Output"))

        ### create list of DGE objects for total, input, and output
        lst = list()
        lst[["Total"]]  = DGEList(counts=mat_tot, group = dat_col$Group)
        lst[["Input"]]  = DGEList(counts=mat_inp, group = rep("Input",  ncol(mat_inp)))
        lst[["Output"]] = DGEList(counts=mat_out, group = rep("Output", ncol(mat_out)))
        lst_edger = lst

        ### filter counts using filterByExpr from edgeR
        dge = lst_edger[["Total"]]
        idx = filterByExpr(dge)

        ### Check: index matched when filtering
        x = names(idx)
        y = row.names(mat_tot)
        if (!all(x == y)) {stop("Error index")}

        ### if passed: filter matrix
        mat_cnt = mat_tot[idx,]

        ### show progress:
        txt_fpath  = txt_fpath_inp
        txt_fname  = basename(txt_fpath)
        txt_prefix = fun_get_prefix(txt_fname)
        
        cat("\n====================\n")
        cat("Region:", txt_region, "\n")
        cat("Assay: ", txt_assay,  "\n")
        cat("Prefix:", txt_prefix, "\n")
        cat("FPath: ", txt_fpath,  "\n")
        cat("FName: ", txt_fname,  "\n")
        cat("Import Counts:\n")
        cat("#Rows (Before filter):", nrow(dat_cnt), "\n")
        cat("#Rows (After  filter):", nrow(mat_cnt), "\n")
        cat("\n")
        flush.console()

        ### create dds object
        cat("Apply DESeq2: ~Group", "\n")
        dds = DESeqDataSetFromMatrix(
            countData = mat_cnt, 
            colData   = dat_col, 
            design    = ~Group)
        
        ### ensure Input group is set as reference
        dds$Group <- relevel(dds$Group, ref = "Input")

        ### run deseq analysis
        dds = DESeq(dds)

        ### save deseq object
        txt_fdiry = dirname(txt_fpath_inp)
        txt_fname = paste("data", "deseq2", txt_prefix, "rds", sep=".")
        txt_fpath = file.path(txt_fdiry, txt_fname)
        
        obj = dds
        saveRDS(obj, txt_fpath)

        ### save DGEList object
        txt_fdiry = dirname(txt_fpath_inp)
        txt_fname = paste("data", "edger", txt_prefix, "rds", sep=".")
        txt_fpath = file.path(txt_fdiry, txt_fname)
        
        obj = lst_edger
        saveRDS(obj, txt_fpath)
    }
}


Region: atac_ENCFF333TAT 
Assay:  MPRA_Tiling_K562_Tewhey_Hannah 
Prefix: OL13 
FPath:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/atac_ENCFF333TAT/MPRA_Tiling_K562_Tewhey_Hannah/overlap_count/summary/data.count_column.raw.OL13.rds 
FName:  data.count_column.raw.OL13.rds 
Import Counts:
#Rows (Before filter): 20 
#Rows (After  filter): 20 

Apply DESeq2: ~Group 


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




Region: atac_ENCFF333TAT 
Assay:  MPRA_Tiling_K562_Tewhey_Hannah 
Prefix: OL43 
FPath:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/atac_ENCFF333TAT/MPRA_Tiling_K562_Tewhey_Hannah/overlap_count/summary/data.count_column.raw.OL43.rds 
FName:  data.count_column.raw.OL43.rds 
Import Counts:
#Rows (Before filter): 337 
#Rows (After  filter): 337 

Apply DESeq2: ~Group 


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing




Region: atac_ENCFF333TAT 
Assay:  MPRA_Tiling_K562_Tewhey_Hannah 
Prefix: OL45 
FPath:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/atac_ENCFF333TAT/MPRA_Tiling_K562_Tewhey_Hannah/overlap_count/summary/data.count_column.raw.OL45.rds 
FName:  data.count_column.raw.OL45.rds 
Import Counts:
#Rows (Before filter): 834 
#Rows (After  filter): 834 

Apply DESeq2: ~Group 


...

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




Region: fcc_astarr_macs_input_union 
Assay:  MPRA_Tiling_K562_Tewhey_Hannah 
Prefix: OL43 
FPath:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_union/MPRA_Tiling_K562_Tewhey_Hannah/overlap_count/summary/data.count_column.raw.OL43.rds 
FName:  data.count_column.raw.OL43.rds 
Import Counts:
#Rows (Before filter): 394 
#Rows (After  filter): 394 

Apply DESeq2: ~Group 


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




Region: fcc_astarr_macs_input_union 
Assay:  MPRA_Tiling_K562_Tewhey_Hannah 
Prefix: OL45 
FPath:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/fcc_astarr_macs_input_union/MPRA_Tiling_K562_Tewhey_Hannah/overlap_count/summary/data.count_column.raw.OL45.rds 
FName:  data.count_column.raw.OL45.rds 
Import Counts:
#Rows (Before filter): 1306 
#Rows (After  filter): 1306 

Apply DESeq2: ~Group 


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

